In [ ]:
import matplotlib.pyplot as plt
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool                                              # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import accuracy_score
from joblib import Parallel,delayed
import multiprocessing as mp
from numba import njit
import shutil
from tqdm.notebook import tqdm
from tensorflow import Tensor

In [ ]:
#Unpacking the zip file
shutil.unpack_archive('eye_gender_data.zip', 'eye_gender_data')

In [ ]:
#Here, we load the labels from their csv file
labels = pd.read_csv("eye_gender_data/Training_set.csv")
#For every label, we extract the file_path to that particular image
file_paths = [[fname, 'eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
data = pd.merge(images, labels, how = 'inner', on = 'filename')

#To avoid variability in image size, as we're using a basic CNN model, we fix the image size to 100. 
#This is what works best for this dataset. Other sizes can be taken too.
#However, once the size is fixed, it should be used for all the training and test data. Our CNN can't accept different image sizes
image_size = 100
#Resize every image to our pre-set image size
X = np.array([cv2.resize(cv2.imread(data['filepaths'][i], cv2.IMREAD_GRAYSCALE),(image_size,image_size)).reshape(image_size,image_size,1) for i in range(len(images))])
#Mapping labels to binary
#0 for male and 1 for female
Y = np.array([0.0 if (data['label'][i]) == 'male' else 1.0 for i in range(len(images))])

In [ ]:
#Let's examine our training set. Here we plot four random images from our training set
random_images = [np.random.randint(0, len(X)) for i in range(4)]
f, axarr = plt.subplots(2,2)

axarr[0,0].set_title(str(Y[random_images[0]])+" : "+labels['label'][random_images[0]])
axarr[0,0].imshow(X[random_images[0]].reshape(image_size, image_size))
axarr[0, 0].set_axis_off()

axarr[0,1].set_title(str(Y[random_images[1]])+" : "+labels['label'][random_images[1]])
axarr[0,1].imshow(X[random_images[1]].reshape(image_size, image_size), cmap='gray')
axarr[0, 1].set_axis_off()

axarr[1,0].set_title(str(Y[random_images[2]])+" : "+labels['label'][random_images[2]])
axarr[1,0].imshow(X[random_images[2]].reshape(image_size, image_size), cmap='gray')
axarr[1, 0].set_axis_off()

axarr[1,1].set_title(str(Y[random_images[3]])+" : "+labels['label'][random_images[3]])
axarr[1,1].imshow(X[random_images[3]].reshape(image_size, image_size), cmap='gray')
axarr[1, 1].set_axis_off()

In [ ]:
#Here, we define our Data Augmentation
def augment(inputs: Tensor) -> Tensor:
    out = layers.experimental.preprocessing.RandomFlip("horizontal")(inputs)
    out = layers.experimental.preprocessing.RandomRotation(0.1)(out)
    out = layers.experimental.preprocessing.RandomZoom(0.5)(out)
    out = layers.experimental.preprocessing.RandomCrop(image_size,image_size)(out)
    return out


In [ ]:
#Let's now look at various augmented examples of a single image
rand_img = np.random.randint(0, len(X))
print("Image Label : "+str(Y[rand_img])+" : "+labels['label'][rand_img])
image = X[rand_img]
plt.figure(figsize=(15,10))
for i in range(9):
    aug_image = np.array(augment(image))
    ax = plt.subplot(3,3,i+1)
    plt.imshow(aug_image.reshape(image_size, image_size), cmap='gray')
    plt.axis('off') 

In [ ]:
#Importing the required Keras Layers
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

#We define relu_bn as a combo of ReLU + BatchNorm layers
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

#We define a residual_block which uses a skip connection here
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

#Here, we combine our final model architecture
def create_model():              
    inputs = Input(shape=(image_size, image_size, 1))
    
    num_filters = 64
    t = inputs
    t = augment(t)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
            t = Dropout(0.3)(t)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(64, activation='relu')(t)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs, outputs)
    
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),metrics=['accuracy'])    

    return model

In [ ]:
#Here we create our model and train it. Increase no of epochs for more accuracy
model = create_model()
history = model.fit(X, Y, epochs=10, batch_size=32)

In [ ]:
#We save our model as my_model
model.save("my_model")

In [ ]:
#We save an image of our model's architecture
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')